1. Criar uma classe que aceite os dados.
2. Necessario ser um Dataframe; aceitar y_pred e y; a coluna de timestamp e o período em dias. Todos campos requeridos.
3. Usar acurácia balanceada ou a diferença das acurácias
4. Criar o intervalo de confiança, criar as bandas com toda a base de dados.
5. Dar a possibilidade de estimar ou não o resultado
6. Definir limiar
7. Estimar os resultados e retornar df de resposta
8. Fazer um gráfico que demonstre a referencia versus estimado

In [686]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
import plotly.express as px
import numpy as np
from scipy.stats import norm
import plotly.graph_objects as go
from sklearn.metrics import f1_score

Dados

In [687]:

# Parâmetros
n_samples = 100000 
n_features = 20  
n_informative = 2   # Número de características informativas
n_redundant = 2    # Número de características redundantes
weights = [0.2, 0.8]  # Desbalanceamento

# Gerar os dados
X, y = make_classification(n_samples=n_samples, 
                           n_features=n_features, 
                           n_informative=n_informative, 
                           n_redundant=n_redundant, 
                           #weights=weights,
                           flip_y=0.05, 
                           random_state=42,
                           )

In [688]:
def generate_datetime_interval(start_date, years, n_samples):
    """
    Função para gerar um intervalo de datas.
    
    Parâmetros:
    - start_date: Data inicial
    - years: Quantidade de anos
    - n_samples: Número de amostras
    
    Retorna:
    - np.array de datas
    """
    np.random.seed(42)

    weeks = years * 52 
    res = np.empty(n_samples, dtype='datetime64[s]')
    date = start_date
    
    for i in range(0, n_samples, n_samples // weeks):
        date_range = pd.date_range(start=date, periods=7, freq='D')
        segment_size = min(n_samples // weeks, n_samples - i)
        res[i:i + segment_size] = np.random.choice(date_range, size=segment_size, replace=True)
        date += timedelta(7)

    return res

In [689]:
# Criar DataFrame
df = pd.DataFrame(X, columns=[f'feature_{i}' for i in range(n_features)])
df['target'] = y
df['datetime'] = generate_datetime_interval(datetime(2021, 1, 1), 4, n_samples)
df_train = df[df["datetime"] < '2024']
df_reference = df[(df["datetime"] >= '2024-01-01') & (df["datetime"] < '2024-07-01')].copy()
df_test = df[(df["datetime"] >= '2024-07-01')].copy()

In [691]:
X_train = df_train[df_train.columns[:-2]]
y_train = df_train["target"]

In [692]:
rf = RandomForestClassifier(random_state=42, oob_score=True, n_jobs=-1, class_weight="balanced")
rf.fit(X_train, y_train)

RandomForestClassifier(class_weight='balanced', n_jobs=-1, oob_score=True,
                       random_state=42)

In [693]:
df_test["prediction"] = rf.predict(df_test[df_test.columns[:-2]])
df_reference["prediction"] = rf.predict(df_reference[df_reference.columns[:-2]])

In [694]:
def calculate_metric_by_period(df, period, target_col, prediction_col, metric=f1_score):
    """
    Função para agrupar o DataFrame por períodos e calcular uma métrica.
    
    Parâmetros:
    - df: DataFrame
    - period: Período de agrupamento (ex: 'W' para semanas, 'M' para meses)
    - target_col: Nome da coluna do alvo
    - prediction_col: Nome da coluna de predição
    - metric: Função métrica (ex: f1_score)
    
    Retorna:
    - DataFrame com a métrica calculada por período
    """
    grouped = df.groupby(pd.Grouper(key='datetime', freq=period)).apply(lambda x: metric(x[target_col], x[prediction_col]))
    return grouped.reset_index(name=f'metric')


In [695]:
result = calculate_metric_by_period(df_reference, 'W', 'target', 'prediction', f1_score)

In [696]:
px.histogram(result["metric"])

In [697]:
def bootstrapping_bca(data, confidence_level, statistic=np.mean, num_iterations=1000):
    """
    Calculates the bias-corrected and accelerated (BCa) bootstrap confidence interval for the given data.

    Parameters:
    - data (list or numpy array): Sample data.
    - confidence_level (float): Desired confidence level (e.g., 0.95 for 95%).
    - statistic (function): Statistical function to apply to the data. Default is np.mean.
    - num_iterations (int): Number of bootstrap resamples to perform. Default is 1000.

    Returns:
    - tuple: A tuple containing the lower and upper bounds of the BCa confidence interval.
    """
    np.random.seed(42)
    n = len(data)

    def generate_acceleration(data):
        """
        Calculates the jackknife resampling and returns the acceleration.

        Parameters:
        - data (list or numpy array): Sample data.

        Returns:
        - float: Acceleration value calculated from jackknife samples.
        """
        jackknife = np.zeros(n)
        
        for i in range(n):
            jackknife_sample = np.concatenate([data[:i], data[i+1:]])  # Remove o elemento na posição i
            jackknife[i] = statistic(jackknife_sample)
        
        jackknife_mean = np.mean(jackknife)
        jackknife_diffs = jackknife - jackknife_mean
        acceleration = np.sum(jackknife_diffs ** 3) / (6.0 * (np.sum(jackknife_diffs ** 2) ** 1.5))
        
        return acceleration

    def calculate_bootstrap_statistics(data, statistic, num_iterations):
        """
        Performs bootstrap resampling on the given data and calculates the specified statistic for each resample.

        Parameters:
        - data (list or numpy array): Sample data.
        - statistic (function): Statistical function to apply to the data.
        - num_iterations (int): Number of bootstrap resamples to perform. Default is 1000.

        Returns:
        - numpy array: Array of calculated statistics for each bootstrap resample.
        """
        sample_statistics = np.zeros(num_iterations)
        
        for i in range(num_iterations):
            resample = np.random.choice(data, size=n, replace=True)
            sample_statistics[i] = statistic(resample)
        
        return sample_statistics

    # Bootstrap resampling 
    sample_statistics = calculate_bootstrap_statistics(data, statistic, num_iterations)
    
    # Bias correction
    observed_stat = statistic(data)
    
    bias = np.sum(sample_statistics < observed_stat) / num_iterations
    
    # Jackknife resampling
    acceleration = generate_acceleration(data)
    
    # Calculating BCa interval
    z0 = norm.ppf(bias)
    z_alpha = norm.ppf((1 + confidence_level) / 2)
    
    # Applying BCa formula for percentiles
    lower_bound_percentile = norm.cdf(z0 + (z0 - z_alpha) / (1 - acceleration * (z0 - z_alpha)))
    upper_bound_percentile = norm.cdf(z0 + (z0 + z_alpha) / (1 + acceleration * (z0 + z_alpha)))
    
    # Calculate lower and upper bounds from the percentiles
    lower_bound = np.percentile(sample_statistics, lower_bound_percentile * 100)
    upper_bound = np.percentile(sample_statistics, upper_bound_percentile * 100)
    
    return lower_bound, upper_bound



In [698]:
def plot_performance_by_time(reference, analysis):
    # Criando o gráfico
    fig = go.Figure()

    # Adicionando a linha dos valores da métrica
    fig.add_trace(go.Scatter(
        x=analysis["datetime"], 
        y=analysis["metric"], 
        mode='lines+markers', 
        name='Métrica', 
                            )
                            )

    # Adicionando a faixa de intervalo de confiança fixa (shaded area)
    fig.add_trace(go.Scatter(x=analysis["datetime"], 
                            y=[reference["ci_lower"], reference["ci_upper"]],
                            fill='toself', 
                            fillcolor='rgba(0, 100, 255, 0.2)',  # Azul com opacidade
                            line=dict(color='rgba(255,255,255,0)'),  # Sem linha na borda
                            name='Intervalo de Confiança Fixo'))

    fig.add_hrect(y0=reference["ci_lower"], y1=reference["ci_upper"], line_width=0, fillcolor='lightblue', opacity=0.5)

    fig.add_hline(y=reference["upper_threshold"], line_dash='dash', line_color='firebrick', opacity=0.5)

    fig.add_hline(y=reference["lower_threshold"], line_dash='dash', line_color='firebrick', opacity=0.5)

    fig.add_hline(y=reference["mean"], line_dash='dash', line_color='darkslateblue', opacity=0.3)

    # Adicionando rótulos e título
    fig.update_layout(
        title="Métrica ao Longo do Tempo com Intervalo de Confiança Fixo",
        xaxis_title="Tempo",
        yaxis_title="Métrica",
        showlegend=True,
    )

    # Exibindo o gráfico
    return fig.show()


In [699]:
def generate_reference_metrics(df, period, target_col, prediction_col, statistic, confidence_level=0.997):
    """
    Calcula métricas de referência para um DataFrame agrupado por um período específico e retorna intervalos de confiança e limiares.

    Parâmetros:
    - df (pd.DataFrame): DataFrame contendo os dados.
    - period (str): Período de agrupamento (ex: 'W' para semanas).
    - target_col (str): Nome da coluna do alvo (rótulo verdadeiro).
    - prediction_col (str): Nome da coluna de predição.
    - statistic (function): Função estatística para calcular a métrica.
    - confidence_level (float): Nível de confiança para o intervalo de confiança. Default é 0.997.

    Retorna:
    - dict: Dicionário contendo o intervalo de confiança, a média e os limiares superior e inferior da métrica calculada.
    """
    # Calcular as métricas agrupadas por período
    metrics_by_period = calculate_metric_by_period(df, period, target_col, prediction_col, statistic)
    
    # Calcular o intervalo de confiança usando bootstrapping
    ci_lower, ci_upper = bootstrapping_bca(metrics_by_period['metric'], confidence_level=confidence_level)
    
    # Calcular a média estimada da métrica
    estimated_mean_statistic = np.mean(metrics_by_period["metric"])
    
    # Calcular o desvio padrão
    std_deviation = metrics_by_period["metric"].std()
    
    # Calcular os limiares
    upper_threshold = estimated_mean_statistic + (std_deviation * 3)
    lower_threshold = estimated_mean_statistic - (std_deviation * 3)

    # Criar o dicionário de resultados
    results = {
        "ci_lower": ci_lower,
        "ci_upper": ci_upper,
        "mean": estimated_mean_statistic,
        "lower_threshold": lower_threshold,
        "upper_threshold": upper_threshold
    }

    return results


In [710]:
reference_metrics = generate_reference_metrics(df_reference, "W", "target", "prediction", f1_score)

In [711]:
result_reference = calculate_metric_by_period(df_reference, 'W', 'target', 'prediction')

In [712]:
plot_performance_by_time(reference_metrics, result_reference)

In [713]:
result_test = calculate_metric_by_period(df_test, 'W', 'target', 'prediction', f1_score)
plot_performance_by_time(reference_metrics, result_test)

In [704]:
reference_metrics

{'ci_lower': 0.8489823744269228,
 'ci_upper': 0.8623936275103546,
 'mean': 0.856128632809579,
 'lower_threshold': 0.8189637750591849,
 'upper_threshold': 0.893293490559973}